<a href="https://colab.research.google.com/github/balakumar-dataengineer/testrepo/blob/master/Pyspark_UDF(User_define_functions).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('pyspark') \
                          .master('local') \
                          .getOrCreate()

print(spark)

In [2]:
data=[
    (1,'bala',2000,3000),
    (2,'kumar',3000,40000),
    (3,'balakumar',1500000,500000)
]

schema=['id','name','fixedPay','variablePay']

df=spark.createDataFrame(data,schema)
df.show()
df.printSchema()

+---+---------+--------+-----------+
| id|     name|fixedPay|variablePay|
+---+---------+--------+-----------+
|  1|     bala|    2000|       3000|
|  2|    kumar|    3000|      40000|
|  3|balakumar| 1500000|     500000|
+---+---------+--------+-----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- fixedPay: long (nullable = true)
 |-- variablePay: long (nullable = true)



In [21]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

def totalPay(f,v):
  return f+v

totalPayFun=udf(totalPay,StringType())

df.select('id','name','fixedPay','variablePay',totalPayFun('fixedPay','variablePay').alias('totalPay')).show()


+---+---------+--------+-----------+--------+
| id|     name|fixedPay|variablePay|totalPay|
+---+---------+--------+-----------+--------+
|  1|     bala|    2000|       3000|    5000|
|  2|    kumar|    3000|      40000|   43000|
|  3|balakumar| 1500000|     500000| 2000000|
+---+---------+--------+-----------+--------+



In [12]:
def totPay(df):
  return df.withColumn('totalPay',df['fixedPay']+df['variablePay'])

df=totPay(df)
df.show()

+---+---------+--------+-----------+--------+
| id|     name|fixedPay|variablePay|totalPay|
+---+---------+--------+-----------+--------+
|  1|     bala|    2000|       3000|    5000|
|  2|    kumar|    3000|      40000|   43000|
|  3|balakumar| 1500000|     500000| 2000000|
+---+---------+--------+-----------+--------+



In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(returnType=StringType())
def toPay(f,v):
  return f+v

df.select('id','name','fixedPay','variablePay',toPay('fixedPay','variablePay').alias('topay')).show()




+---+---------+--------+-----------+-------+
| id|     name|fixedPay|variablePay|  topay|
+---+---------+--------+-----------+-------+
|  1|     bala|    2000|       3000|   5000|
|  2|    kumar|    3000|      40000|  43000|
|  3|balakumar| 1500000|     500000|2000000|
+---+---------+--------+-----------+-------+



In [19]:
df=df.drop('totalPay')
df.show()

+---+---------+--------+-----------+
| id|     name|fixedPay|variablePay|
+---+---------+--------+-----------+
|  1|     bala|    2000|       3000|
|  2|    kumar|    3000|      40000|
|  3|balakumar| 1500000|     500000|
+---+---------+--------+-----------+



In [20]:
df.createOrReplaceTempView('employee')

spark.sql('select*From employee').show()

+---+---------+--------+-----------+
| id|     name|fixedPay|variablePay|
+---+---------+--------+-----------+
|  1|     bala|    2000|       3000|
|  2|    kumar|    3000|      40000|
|  3|balakumar| 1500000|     500000|
+---+---------+--------+-----------+



In [24]:
def payTotal(x,y):
  return x+y

spark.udf.register('topayfun',payTotal,IntegerType())

spark.sql('select id,name,fixedPay,variablePay,topayfun(fixedPay,variablePay) as Total from employee').show()

+---+---------+--------+-----------+-------+
| id|     name|fixedPay|variablePay|  Total|
+---+---------+--------+-----------+-------+
|  1|     bala|    2000|       3000|   5000|
|  2|    kumar|    3000|      40000|  43000|
|  3|balakumar| 1500000|     500000|2000000|
+---+---------+--------+-----------+-------+

